In [1]:
import os

import cv2
import tensorflow as tf
import numpy as np
import pandas as pd

from model.destr_model import ObjDetSplitTransformer, train_one_step
from utils.data_loader import load_data_tfrecord

2024-07-05 01:10:31.947925: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 01:10:31.949975: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-07-05 01:10:31.956947: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:479] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-05 01:10:31.970402: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:10575] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-05 01:10:31.970425: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1442] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-05 01:10:31.979589: I tensorflow/core/platform/cpu_feature_guard.cc:

In [2]:
num_cls = 8
EPOCH_NUMS = 10
BATCH_SIZE = 8
checkpoint_dir = '/workspace/models/checkpoints'

load_from_ckpt = False

In [3]:
destr_block = ObjDetSplitTransformer(input_shape=(224, 224, 3), num_cls=num_cls)

img = tf.keras.Input(shape=(224, 224, 3), dtype=tf.float32)
cls_output, reg_output, total_proposals = destr_block(img)

model = tf.keras.Model(inputs=img, outputs=[cls_output, reg_output, total_proposals])

In [4]:
# Load the weights from latest checkpoint

if load_from_ckpt:
    checkpoint = tf.train.Checkpoint(model=model)
    status = checkpoint.restore(tf.train.latest_checkpoint(checkpoint_dir))

In [5]:
local_dir = '/media/daniel/DatasetIMDB/imdb_chunks'
container_dir = '/workspace/data/tfrecords'

dataset = load_data_tfrecord(path_to_tfrecord=local_dir)

In [6]:
loss_history = []
optimizers = {'mini_det': tf.keras.optimizers.Adam(learning_rate=0.00001), 'destr': tf.keras.optimizers.Adam(learning_rate=0.00001)}
optimizer = tf.keras.optimizers.Adam(learning_rate=0.00001)

for epoch_idx in range(10, EPOCH_NUMS+10):
    total_md_loss, total_loss, cnt = 0, 0, 0
    for batch in dataset:
        logits, coord, label, oh_label = batch
        
        mini_det_loss, model_loss = train_one_step(
                model, optimizer, 
                tf.reshape(tf.cast(tf.io.decode_raw(logits, tf.uint8), tf.float32), (-1, 224, 224, 3)), 
                tf.concat([label[..., tf.newaxis], oh_label, coord], axis=-1)
            )
        total_md_loss += mini_det_loss.numpy()
        total_loss += model_loss.numpy()
        cnt += 1
        break
        

    loss_history.append((total_md_loss / cnt, total_loss / cnt))
    # Save parameters of model each epoch
    #checkpoint = tf.train.Checkpoint(model=model)
    #checkpoint_prefix = os.path.join(checkpoint_dir, f'ckpt_{epoch_idx}')
    #checkpoint.save(file_prefix=checkpoint_prefix)
    
    print(f'{epoch_idx+1}: {loss_history[-1]}')

W0000 00:00:1720113088.826713  128849 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 4300 num_cores: 12 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } environment { key: "eigen" value: "3.4.90" } l1_cache_size: 32768 l2_cache_size: 524288 l3_cache_size: 33554432 memory_size: 268435456 } outputs { dtype: DT_FLOAT shape { unknown_rank: true } }
W0000 00:00:1720113088.834213  128849 op_level_cost_estimator.cc:699] Error in PredictCost() for the op: op: "Softmax" attr { key: "T" value { type: DT_FLOAT } } inputs { dtype: DT_FLOAT shape { unknown_rank: true } } device { type: "CPU" vendor: "AuthenticAMD" model: "241" frequency: 4300 num_cores: 12 environment { key: "cpu_instruction_set" value: "AVX SSE, SSE2, SSE3, SSSE3, SSE4.1, SSE4.2" } enviro

11: (0.9955236315727234, 1.589211106300354)
12: (0.9217894077301025, 1.532339334487915)
13: (0.9226610660552979, 1.2473564147949219)
14: (0.9467150568962097, 0.8409180641174316)
15: (0.820968747138977, 1.031234860420227)
16: (0.8413122296333313, 0.9218123555183411)
17: (0.7549859881401062, 0.8561890125274658)
18: (1.0512146949768066, 0.9150417447090149)
19: (0.7037288546562195, 0.7918254733085632)
20: (0.6137189269065857, 0.9108594059944153)


In [7]:
#model.load_weights('/workspace/models/destr_20')
#weights = model.get_weight()
#model.set_weights(weights)